# Modelo Aditivos Generalizados

- Son utilizados principalmente para combatir al underfitting (cuando nuestro modelo es demasiado
rígido, o tiene mucho sesgo/bias)
- Soluciona problemas cuando nuestro modelo genera una linea recta, y en verdad nuestro datos
tienen una curvatura (ej: exponenciales)

## Soluciones a la nolinealidad

- Modelo Lineal: Presenta alto sesgo, pero es fácil de interpretar
- Polinomial: Se acoplan harto a los datos de entrenamiento, pero tienen una chance de generalización
mucho más baja
- GLM: Suaviza/Genera curvas en el modelo hecho. Deja a las variables del modelo en forma lineal
aún así!

## Elementos básicos de GAM - Solución propuesta por GAM

Aquí se utilizan funciones en cada una de las variables. Por ejemplo:

Modelo con 4 variables (A, B, C y D)

y = intercepto + f(A) + f(B) + f(C) + f(D)

Donde f son las funciones que se aplican a cada una de las variables. Por ejemplo: Función identidad
(la variable se deja de la misma forma), función cuadrada (x elevado a 2), función cúbica, función
logaritmica, etc... . Esto hace que nuestro modelo en si siga siendo lineal, pero se introducen
curvaturas a la variable dependiente!